In [1]:
import requests;


In [2]:
with open(r"C:\Users\Saksham\Desktop\CC_task\pokemon.txt","r") as pk:
    content = pk.readlines()
    print(content)

['Amoonguss\n', 'Gyarados\n', '143\n', 'Volcarona\n', 'Wooper\n', 'Greninja\n', '231\n', 'Charizard\n', 'Sceptile\n', '681\n', 'Blaziken\n', 'Mudkip\n', 'Rowlet\n', '292\n', 'Wynaut\n', 'Incineroar\n', 'Spheal\n', 'Buzzwole\n', 'Salamence\n', '778\n', 'Metagross\n', 'Rayquaza\n', 'Falinks\n', 'Infernape\n', 'Piplup\n', 'Dragapult\n', 'Darkrai\n', 'Gholdengo\n', '501\n', 'Walking-Wake\n', 'Milotic\n', 'Bulbasaur\n', 'Gengar\n', 'Umbreon\n', 'Garchomp\n', 'Chandelure\n', 'Lucario\n', 'Flygon\n', 'Scizor\n', 'Squirtle\n', 'Dragonite\n', 'Luxray\n', 'Lugia\n', 'Noivern\n', 'Ceruledge\n', 'Haxorus\n', 'Psyduck\n', 'Shedinja\n', 'Annihilape\n', 'Magikarp']


In [3]:
my_dict={}

In [4]:
for i in range(0,50): #50 is the length
    name_id = content[i].strip()
    r1=requests.get(f"https://pokeapi.co/api/v2/pokemon/{name_id}/")
    name=r1.json()["name"]
    idno=r1.json()["id"]
    type_=[]
    abilities=[]
    for ability in r1.json()["abilities"]:
            abilities.append(ability['ability']['name'])
    for types in r1.json()["types"]:
            type_.append(types['type']['name'])
    r2=requests.get(f"https://pokeapi.co/api/v2/pokemon-species/{name_id}/")
    legendary=r2.json()["is_legendary"]
    mythical=r2.json()["is_mythical"]
    temp_dict={"id":idno,"is_legendary":legendary,"is_mythical":mythical,"abilities":abilities,"types":type_}
    my_dict[name]=temp_dict

In [5]:
damage_array={
    "attackers": {},
    "defenders": {}
}
pokemon_types = [
    "normal",
    "fire",
    "water",
    "electric",
    "grass",
    "ice",
    "fighting",
    "poison",
    "ground",
    "flying",
    "psychic",
    "bug",
    "rock",
    "ghost",
    "dragon",
    "dark",
    "steel",
    "fairy"
]
for j in range(1,19):
    r3=requests.get(f"https://pokeapi.co/api/v2/type/{j}/")
    name=r3.json()['name']
    for k in (r3.json()['damage_relations']):
            temp_attacker={}
            temp_defender={}
            for ele1 in r3.json()['damage_relations']["double_damage_from"]:
                        temp_defender[ele1["name"]]=2
            for ele1 in r3.json()['damage_relations']["double_damage_to"]:
                        temp_attacker[ele1["name"]]=2
            for ele1 in r3.json()['damage_relations']["half_damage_from"]:
                        temp_defender[ele1["name"]]=0.5
            for ele1 in r3.json()['damage_relations']["half_damage_to"]:
                        temp_attacker[ele1["name"]]=0.5
            for ele1 in r3.json()['damage_relations']["no_damage_from"]:
                        temp_defender[ele1["name"]]=0
            for ele1 in r3.json()['damage_relations']["no_damage_to"]:
                        temp_attacker[ele1["name"]]=0
    if (temp_attacker):    
        for ele in pokemon_types:
                    if ele not in temp_attacker.keys():
                            temp_attacker[ele]=1
        damage_array["attackers"].update({name:temp_attacker})
    if (temp_defender):
            for ele in pokemon_types:
                    if ele not in temp_defender.keys():
                            temp_defender[ele]=1
            damage_array["defenders"].update({name:temp_defender})
damage_array


{'attackers': {'normal': {'rock': 0.5,
   'steel': 0.5,
   'ghost': 0,
   'normal': 1,
   'fire': 1,
   'water': 1,
   'electric': 1,
   'grass': 1,
   'ice': 1,
   'fighting': 1,
   'poison': 1,
   'ground': 1,
   'flying': 1,
   'psychic': 1,
   'bug': 1,
   'dragon': 1,
   'dark': 1,
   'fairy': 1},
  'fighting': {'normal': 2,
   'rock': 2,
   'steel': 2,
   'ice': 2,
   'dark': 2,
   'flying': 0.5,
   'poison': 0.5,
   'bug': 0.5,
   'psychic': 0.5,
   'fairy': 0.5,
   'ghost': 0,
   'fire': 1,
   'water': 1,
   'electric': 1,
   'grass': 1,
   'fighting': 1,
   'ground': 1,
   'dragon': 1},
  'flying': {'fighting': 2,
   'bug': 2,
   'grass': 2,
   'rock': 0.5,
   'steel': 0.5,
   'electric': 0.5,
   'normal': 1,
   'fire': 1,
   'water': 1,
   'ice': 1,
   'poison': 1,
   'ground': 1,
   'flying': 1,
   'psychic': 1,
   'ghost': 1,
   'dragon': 1,
   'dark': 1,
   'fairy': 1},
  'poison': {'grass': 2,
   'fairy': 2,
   'poison': 0.5,
   'ground': 0.5,
   'rock': 0.5,
   'ghost': 

In [6]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route("/")
def damage_endpoint():
    attacker = request.args.get("attacker")
    defender = request.args.get("defender")
    # Case 1: Both missing
    if not attacker and not defender:
        return jsonify({"error": "Provide attacker and defender"}), 400
    
    # Case 2: Only defender given
    elif not attacker:
        try:
            data = damage_array["defenders"][defender]
            return jsonify({
                "defender": defender,
                "data": data
            })
        except KeyError:
            return jsonify({"error": f"No data for defender {defender}"}), 404
    
    # Case 3: Only attacker given
    elif not defender:
        try:
            data = damage_array["attackers"][attacker]
            return jsonify({
                "attacker": attacker,
                "data": data
            })
        except KeyError:
            return jsonify({"error": f"No data for attacker {attacker}"}), 404
    
    # Case 4: Both attacker and defender given
    else:
        try:
            result = damage_array["attackers"][attacker][defender]
            return jsonify({
                "attacker": attacker,
                "defender": defender,
                "data": result
            })
        except KeyError:
            return jsonify({"error": f"No data for {attacker} vs {defender}"}), 404

In [10]:
from threading import Thread
def run_app():
    app.run(port=5000)

thread = Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [11]:
from flask import render_template_string,request ,jsonify


app = Flask(__name__)

# Your local damage server (make sure it's running separately)
LOCAL_DAMAGE_SERVER = "http://127.0.0.1:5000"  
POKEAPI_BASE = "https://pokeapi.co/api/v2/pokemon/"

# Simple HTML template with form + results
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Pokemon Type Checker</title>
</head>
<body>
    <h1>Pokemon Type Checker</h1>
    <form method="post" action="/pokemon">
        <label>Enter Pokemon name:</label>
        <input type="text" name="name">
        <input type="submit" value="Search">
    </form>

    {% if result %}
        <h2>Result for {{ result.pokemon }}</h2>
        <p><b>Types:</b> {{ result.types }}</p>
        <h3>Damage Relations:</h3>
        <pre>{{ result.damage_relations | tojson(indent=2) }}</pre>
        {% if result.types|length == 1 %}
            <p><b>Immune against: </b>{{result.immune}} </p>
            <p><b>Weakness against: </b>{{result.weakness}} </p>
        {% elif result.types|length == 2 %}
            <p><b>Immune against: </b>{{result.immune2}} </p>
            <p><b>WeaknessX2 against: </b>{{result.weakness_2}} </p>
            <p><b>WeaknessX$ against: </b>{{result.weakness_4}} </p>
        {% else %}
            <p><b>Types:</b> None found</p>
        {% endif %}
    {% endif %}
    
</body>
</html>
"""

@app.route("/", methods=["GET"])
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route("/pokemon", methods=["POST"])
def get_pokemon():
    pokemon_name = request.form["name"].lower()

    # Step 1: Query PokéAPI
    poke_url = f"{POKEAPI_BASE}{pokemon_name}"
    poke_response = requests.get(poke_url)

    if poke_response.status_code != 200:
        return render_template_string(HTML_TEMPLATE, result={"pokemon": pokemon_name, "types": [], "damage_relations": {"error": "Pokémon not found"}})

    poke_data = poke_response.json()
    types = [t["type"]["name"] for t in poke_data["types"]]

    # Step 2: Query local damage server for each type
    damage_data = {}
    for t in types:
        local_response = requests.get(f"{LOCAL_DAMAGE_SERVER}?defender={t}")
        if local_response.status_code == 200:
            damage_data[t] = local_response.json()
        else:
            damage_data[t] = {"error": "No data available"}

    # Step 3: Return result
    result = {
        "pokemon": pokemon_name,
        "types": types,
        "damage_relations": damage_data
    }
    weakness=[]
    immune=[]
    overall2={}
    weakness_2=[]
    weakness_4=[]
    immune2=[]
    if len(result["types"])==1:
            overall=result["damage_relations"][types[0]]["data"]
            for ele in overall:
                 if overall[ele]==2:
                      weakness.append(ele)
                 elif overall[ele]==0:
                      immune.append(ele)
            result["immune"]=immune
            result["weakness"]=weakness

    else:
            for t in result["damage_relations"][types[0]]["data"]:
                    val1 = result["damage_relations"][types[0]]["data"][t]
                    val2 = result["damage_relations"][types[1]]["data"][t]
                    overall2[t] = val1 * val2
            for ele in overall2:
                if overall2[ele]==2:
                     weakness_2.append(ele)
                elif overall2[ele]==4:
                    weakness_4.append(ele)
                elif overall2[ele]==0:
                     immune2.append(ele)
            result["immune2"]=immune2
            result["weakness_2"]=weakness_2
            result["weakness_4"]=weakness_4
    return render_template_string(HTML_TEMPLATE, result=result)



In [12]:
def run_app():
    app.run(port=5001)

thread = Thread(target=run_app)
thread.start()

 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
